### SPLIT AND REDUCTION OF THE ORIGINAL DATASET

In [2]:
import pandas as pd
import sqlite3
from datetime import datetime


input_file = "itineraries.csv"
chunksize = 10000000  

def get_month(date_str):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").month
    except:
        return None

outputs = {
    "all": {
        "first_file": True,
        "csv_name": "file_all_LAX_data.csv",
        "table": "all_data",
        "data": []
    },
    "august": {
        "first_file": True,
        "csv_name": "file_TEST_LAX_august.csv",
        "table": "august_data",
        "data": []
    },
    "apr_jul": {
        "first_file": True,
        "csv_name": "file_TRAINING__LAX_april_july.csv",
        "table": "april_july_data",
        "data": []
    }
}

conn = sqlite3.connect("LAX_data.db")

for chunk in pd.read_csv(input_file, chunksize=chunksize):
    filtered_chunk = chunk[chunk.iloc[:, 3] == "LAX"].copy()
    if filtered_chunk.empty:
        continue

    filtered_chunk.loc[:, "Month"] = filtered_chunk.iloc[:, 2].apply(get_month)

    for key, config in outputs.items():
        if key == "all":
            data_to_save = filtered_chunk.copy()
        elif key == "august":
            data_to_save = filtered_chunk[filtered_chunk["Month"] == 8].copy()
        elif key == "apr_jul":
            data_to_save = filtered_chunk[filtered_chunk["Month"].between(4, 7)].copy()
        else:
            continue

        if not data_to_save.empty:
            data_to_save = data_to_save.drop(columns=["Month"])
            config["data"].append(data_to_save.copy())  

            data_to_save.to_csv(config["csv_name"], mode='a', index=False, header=config["first_file"])
            data_to_save.to_sql(config["table"], conn, if_exists='append', index=False)

            config["first_file"] = False

conn.close()
print("Processing completed.")

Processing completed.


In [3]:
import numpy as np
import pandas as pd

# Concatenate accumulated data from memory
df_train = pd.concat(outputs["apr_jul"]["data"], ignore_index=True)
df_test = pd.concat(outputs["august"]["data"], ignore_index=True)

print(df_train.shape)
print(df_test.shape)

(3400829, 27)
(1742260, 27)


## Missing values, Drop features, De-duplicate

In [5]:
def clean_flight_data(df):
    """
    Cleans the flight data by:
    1. Dropping rows with any missing (NaN) values
    2. Removing rows where specific segment columns are structurally invalid
    3. Removing rows where any part of specific segment columns contains 'None'
    4. Removing rows with non-numeric or negative values
    5. Removing rows with invalid dates
    6. Dropping unnecessary columns
    7. Removing fully duplicated rows
    """

    # 1. Drop rows with any missing (NaN) values
    df_clean = df.dropna()

    # 2. Remove rows where specific segment columns are structurally invalid
    def is_structurally_invalid(val):
        try:
            val = str(val).strip()
            if val == "||":
                return True
            parts = val.split("||")
            if len(parts) == 2:
                return parts[0].strip() == "" or parts[1].strip() == ""
            return False
        except:
            return True

    multi_segment_cols = [
        'segmentsArrivalTimeEpochSeconds',
        'segmentsArrivalTimeRaw',
        'segmentsArrivalAirportCode',
        'segmentsDepartureAirportCode',
        'segmentsAirlineName',
        'segmentsAirlineCode',
        'segmentsEquipmentDescription',
        'segmentsDurationInSeconds',
        'segmentsDistance',
        'segmentsCabinCode'
    ]

    for col in multi_segment_cols:
        df_clean = df_clean[~df_clean[col].apply(is_structurally_invalid)]

    # 3. Remove rows where any part of specific segment columns contains 'None'
    def has_any_none_value(val):
        if pd.isna(val):
            return True
        parts = str(val).split("||")
        return any(p.strip() == "None" for p in parts)

    none_sensitive_cols = [
        'segmentsDistance',
        'segmentsDurationInSeconds',
        'segmentsDepartureTimeEpochSeconds',
        'segmentsArrivalTimeEpochSeconds'
    ]

    for col in none_sensitive_cols:
        df_clean = df_clean[~df_clean[col].apply(has_any_none_value)]
    
    # 4. Remove rows with non-numeric or negative values
    def is_not_valid_number(val):
        try:
            float(val)
            return False
        except:
            return True

    def has_invalid_split_values(val):
        try:
            parts = str(val).split("||")
            for p in parts:
                if p.strip() == "":
                    continue
                if is_not_valid_number(p.strip()) or float(p.strip()) < 0:
                    return True
            return False
        except:
            return True

    numeric_cols = [
        'totalTravelDistance',
        'seatsRemaining',
        'elapsedDays',
        'baseFare'
    ]

    initial_len = len(df_clean)

    invalid_mask = df_clean[numeric_cols].applymap(is_not_valid_number)
    neg_mask = df_clean[numeric_cols] < 0
    combined_invalid = invalid_mask | neg_mask
    to_remove = combined_invalid.any(axis=1)
    df_clean = df_clean[~to_remove]
    removed_numeric = to_remove.sum()

    segment_cols = [
        'segmentsDepartureTimeEpochSeconds',
        'segmentsArrivalTimeEpochSeconds',
        'segmentsDurationInSeconds',
        'segmentsDistance'
    ]

    removed_segments = {}
    for col in segment_cols:
        mask = df_clean[col].apply(has_invalid_split_values)
        count = mask.sum()
        df_clean = df_clean[~mask]
        removed_segments[col] = count
    
    # 5. Remove rows with invalid dates (flightDate and segments*Raw)
    # 5-1. Remove rows with invalid dates in 'flightDate' and 'searchDate'
    #      - Parse using pd.to_datetime(errors='coerce')
    #      - If parsing fails (invalid date), it becomes NaT → remove such rows

    for date_col in ['flightDate', 'searchDate']:
        df_clean['__parsed_date'] = pd.to_datetime(df_clean[date_col], errors='coerce')
        df_clean = df_clean[df_clean['__parsed_date'].notna()].drop(columns='__parsed_date')

    # 5-2. Check if any of the 'segments*Raw' fields contain invalid date parts
    #       - Split values by '||'
    #       - Extract date portion (before 'T')
    #       - Remove row if any extracted date is invalid
    def has_invalid_raw_date(val):
        try:
            parts = str(val).split("||")
            for p in parts:
                if not p.strip():
                    continue
                date_str = p.strip().split("T")[0]
                if pd.to_datetime(date_str, errors="coerce") is pd.NaT:
                    return True
            return False
        except:
            return True  # If parsing fails, treat as invalid

    raw_date_cols = ['segmentsDepartureTimeRaw', 'segmentsArrivalTimeRaw']
    for col in raw_date_cols:
        mask = df_clean[col].apply(has_invalid_raw_date)
        df_clean = df_clean[~mask]

    # 6. Drop unnecessary columns
    cols_to_drop = [
        'totalFare',
        'legId'
        #'isRefundable', -> will remove later
        #'segmentsAirlineName' -> will remove later
        #'segmentsDepartureTimeRaw', -> will remove later
        #'segmentsArrivalTimeRaw', -> will remove later
    ]
    df_clean = df_clean.drop(columns = cols_to_drop, errors = 'ignore')

    # 7. Drop fully duplicated rows (identical in all columns)
    df_clean = df_clean.drop_duplicates()

    return df_clean

In [6]:
df_train = clean_flight_data(df_train)
df_test = clean_flight_data(df_test)

C:\Users\Mats\AppData\Local\Temp\ipykernel_16084\2775977997.py:91: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  invalid_mask = df_clean[numeric_cols].applymap(is_not_valid_number)
C:\Users\Mats\AppData\Local\Temp\ipykernel_16084\2775977997.py:91: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  invalid_mask = df_clean[numeric_cols].applymap(is_not_valid_number)


In [7]:
print(df_train.shape)
print(df_test.shape)

(2689889, 25)
(1383413, 25)


## Destination Airport One-Hot Encoding

In [9]:
from sklearn import preprocessing

# One-Hot encoding
encoder = preprocessing.OneHotEncoder()

# train
encoded_train = pd.DataFrame(
    encoder.fit_transform(df_train[['destinationAirport']]).toarray(), 
    columns=encoder.get_feature_names_out(['destinationAirport']),
    index=df_train.index
)

# test
encoded_test = pd.DataFrame(
    encoder.transform(df_test[['destinationAirport']]).toarray(), 
    columns=encoder.get_feature_names_out(['destinationAirport']),
    index=df_test.index
)

df_train = df_train.join(encoded_train)
df_test = df_test.join(encoded_test)

## Convert Travel Duration to Minutes
Total travel time from departure til arrival

In [11]:
!pip install isodate

In [12]:
import isodate

# Convert duration to minutes using isodate
def convert_duration_to_min(df):
    df['travelDuration_minutes'] = df['travelDuration'].apply(lambda x: isodate.parse_duration(x).total_seconds() / 60)
    return df

df_train = convert_duration_to_min(df_train)
df_test = convert_duration_to_min(df_test)

## Task 3: Convert searchDate and flightDate to Ｎumerical Values


In [14]:
def convert_date_to_numerical(df, reference_min_dates):
    for feature in reference_min_dates:
        df[feature] = pd.to_datetime(df[feature])
        df[f'{feature}_as_int'] = (df[feature] - reference_min_dates[feature]).dt.days + 1
    return df

# Compute training reference dates
ref_dates = {
    'searchDate': pd.to_datetime(df_train['searchDate']).min(),
    'flightDate': pd.to_datetime(df_train['flightDate']).min()
}

df_train = convert_date_to_numerical(df_train, ref_dates)
df_test = convert_date_to_numerical(df_test, ref_dates)

## New variables 

### isHoliday

In [17]:
!pip install holidays

In [18]:
import holidays

# Get US holidays for 2022
us_holidays = holidays.US(years=[2022])

def is_holiday(df):
    # check holiday
    df['is_holiday'] = df['flightDate'].isin(us_holidays)

    # Convert boolean column to binary
    df['is_holiday'] = df['is_holiday'].astype(int)

    return df

df_train = is_holiday(df_train)
df_test = is_holiday(df_test)

C:\Users\Mats\AppData\Local\Temp\ipykernel_16084\1523949206.py:8: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['is_holiday'] = df['flightDate'].isin(us_holidays)
C:\Users\Mats\AppData\Local\Temp\ipykernel_16084\1523949206.py:8: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['is_holiday'] = df['flightDate'].isin(us_holidays)


### isAroundHoliday
+-3 days

In [20]:
def is_around_holiday(df):
    # Add a column to check if flightDate is a holiday or close to one (±3 days)
    df['isAroundHoliday'] = df['flightDate'].apply(
        lambda x: any((x - pd.Timedelta(days=delta)) in us_holidays for delta in range(-3, 4))
    )

    # Convert boolean column to binary
    df['isAroundHoliday'] = df['isAroundHoliday'].astype(int)

    return df

df_train = is_around_holiday(df_train)
df_test = is_around_holiday(df_test)

### dayOfWeek

In [22]:
def day_of_week(df):
    #create dayOfWeek
    df['dayOfWeekNum'] = df['flightDate'].dt.weekday

    # Convert from int32 to int64
    df['dayOfWeekNum'] = df['dayOfWeekNum'].astype('int64')

    return df

df_train = day_of_week(df_train)
df_test = day_of_week(df_test)

### dayOfWeek search day

In [24]:
def day_of_week2(df):
    #create dayOfWeek
    df['SearchdayOfWeekNum'] = df['searchDate'].dt.weekday

    # Convert from int32 to int64
    df['SearchdayOfWeekNum'] = df['SearchdayOfWeekNum'].astype('int64')

    return df

df_train = day_of_week2(df_train)
df_test = day_of_week2(df_test)

### Month of the year

In [26]:
def month_of_year(df):
    # Create Month of the year
    df['monthNum'] = df['flightDate'].dt.month

    # Convert from int32 to int64
    df['monthNum'] = df['monthNum'].astype('int64')

    return df

df_train = month_of_year(df_train)
df_test = month_of_year(df_test)

### Day of the month

In [28]:
def day_of_month(df):
    # Create Month of the year
    df['dayNum'] = df['flightDate'].dt.day

    # Convert from int32 to int64
    df['dayNum'] = df['dayNum'].astype('int64')

    return df

df_train = day_of_month(df_train)
df_test = day_of_month(df_test)

### Days until the flight

In [30]:
def days_until_flight(df):

    df['daysUntilFlight'] = (df['flightDate'] - df['searchDate']).dt.days

    return df

df_train = days_until_flight(df_train)
df_test = days_until_flight(df_test)

### Number of legs

In [32]:
def num_of_legs(df):
    #Create number of legs
    df['num_legs'] = df['segmentsArrivalTimeEpochSeconds'].apply(
        lambda x : len(str(x).split('||')) if pd.notnull(x) else 0
    )

    return df

df_train = num_of_legs(df_train)
df_test = num_of_legs(df_test)

### Departure time and Arrival time

In [34]:
# Function to get first departure hour
def get_first_hour(time_str):
    first_time = time_str.split("||")[0]
    return pd.to_datetime(first_time).hour

# Function to get last arrival hour
def get_last_hour(time_str):
    last_time = time_str.split("||")[-1]
    return pd.to_datetime(last_time).hour

def departure_arrival_time(df):
    # Apply the functions
    df["departureHour"] = df["segmentsDepartureTimeRaw"].apply(get_first_hour)
    df["arrivalHour"] = df["segmentsArrivalTimeRaw"].apply(get_last_hour)

    return df

df_train = departure_arrival_time(df_train)
df_test = departure_arrival_time(df_test)

### Total transfer time

In [36]:
#Create transfer time

def calculate_transfer_time(row):
    dep_times = row['segmentsDepartureTimeRaw'].split('||')
    arr_times = row['segmentsArrivalTimeRaw'].split('||')
    
    # Convert to datetime
    dep_times = [pd.to_datetime(t) for t in dep_times]
    arr_times = [pd.to_datetime(t) for t in arr_times]
    
    # If single leg, no transfer time
    if len(dep_times) <= 1:
        return 0

    # Calculate transfer times (i.e., next dep - prev arr)
    transfer_times = [
        (dep_times[i] - arr_times[i - 1]).total_seconds()
        for i in range(1, len(dep_times))
    ]
    
    # Sum all transfer times
    total_transfer_seconds = sum(transfer_times)
    return total_transfer_seconds / 60  # in minutes

def total_transfer_time(df):
    df['totalTransferTime'] = df.apply(calculate_transfer_time, axis=1)

    return df

df_train = total_transfer_time(df_train)
df_test = total_transfer_time(df_test)

### TotalFlightTime

In [38]:
def total_flight_time(df):
    #Create totalFlightTime
    df['totalFlightTime'] = df['travelDuration_minutes'] - df['totalTransferTime']

    return df

df_train = total_flight_time(df_train)
df_test = total_flight_time(df_test)

### Total distance

In [40]:
#Create total distance

def safe_sum(segment):
    if not isinstance(segment, str):
        return 0
    try:
        return sum(int(s) for s in segment.split('||') if s.isdigit())
    except Exception:
        return 0

def total_distance(df):
    df['total_distance'] = df['segmentsDistance'].apply(safe_sum)

    return df

df_train = total_distance(df_train)
df_test = total_distance(df_test)

### Airlines with the longest distance

In [42]:
#Create Airlines with the longest distance

# Assuming the 'isNonStop' column is a boolean that tells whether the flight is non-stop.
# We also assume the 'segmentsAirlineName' contains the airline names for each leg of the trip.

def get_airline_longest_distance_or_direct(row):
    # Check if the flight is non-stop
    if row['isNonStop']:
        # If it's a direct flight, return the first airline name from segmentsAirlineName
        airlines = row['segmentsAirlineName'].split('||')  # Split airlines by '||'
        return airlines[0] if airlines else None  # Return the first airline if available
    else:
        # If it's not a direct flight, process the segments to find the airline with the longest distance
        if isinstance(row['segmentsDistance'], str) and isinstance(row['segmentsAirlineName'], str):
            distances = row['segmentsDistance'].split('||')
            airlines = row['segmentsAirlineName'].split('||')
            
            # Convert distances to integers, handling non-numeric values
            valid_distances = []
            for distance in distances:
                try:
                    valid_distances.append(int(distance))
                except ValueError:
                    valid_distances.append(None)  # Append None for invalid values
            
            # Check if we have valid distances
            if valid_distances:
                # Find the index of the segment with the longest distance (ignore None values)
                longest_distance_index = max(
                    range(len(valid_distances)), key=lambda i: (valid_distances[i] if valid_distances[i] is not None else -1)
                )
                # Get the airline corresponding to the longest distance
                airline_with_longest_distance = airlines[longest_distance_index]
                return airline_with_longest_distance
            else:
                return None  # If all distances are invalid, return None
        else:
            return None  # If the data is missing or not valid, return None

# Apply the function to create the new 'longestDistanceAirline' column
#df['longestDistanceAirline'] = df.apply(get_airline_longest_distance_or_direct, axis=1)


### Airline frequency-encoding weighted by distance 

In [44]:
import re
from collections import defaultdict

def airline_count_weighted_encoding(df, airline_col='segmentsAirlineName', distance_col='segmentsDistance', sep='||'):
    """
    Encodes each row into airline features by:
    1. Counting how many times each airline appears,
    2. Multiplying by total distance per airline,
    3. Normalizing the row so values sum to 1.
    
    Returns:
        DataFrame with airline columns added.
    """
    
    # Function to parse a single row into airlines and distances
    def parse_row(airline_str, distance_str):
        # Split airlines and distances by the delimiter (e.g., '||')
        airlines = re.split(r'\|+', airline_str) if pd.notnull(airline_str) else []
        raw_distances = re.split(r'\|+', distance_str) if pd.notnull(distance_str) else []

        # Convert distance strings to floats and handle potential errors
        distances = []
        for d in raw_distances:
            try:
                distances.append(float(d.strip())) # Convert distance to float
            except:
                distances.append(0.0) # Handle any invalid distance values by setting them to 0.0
        
        # Ensure both lists are the same length (by trimming the longer list)
        n = min(len(airlines), len(distances))
        return airlines[:n], distances[:n]
    
    # Initialize a list to store the encoded feature rows
    feature_rows = []

    # Set to store all unique airlines encountered
    all_airlines = set()
    
    # Iterate over each row of the dataframe (may takes a long time)
    for _, row in df.iterrows():
        # Parse the airlines and distances for the current row
        airlines, distances = parse_row(row[airline_col], row[distance_col])

        # Dictionaries to store the count and total distance for each airline
        airline_counts = defaultdict(int) # Default to 0 for counts
        airline_distances = defaultdict(float) # Default to 0.0 for distances

        # Accumulate counts and total distance per airline
        for a, d in zip(airlines, distances):
            airline_counts[a] += 1 # Increment the count for this airline
            airline_distances[a] += d # Add the distance to the airline's total
            all_airlines.add(a) # Add the airline to the set of all airlines

        # Compute the weighted values: airline count * total distance
        weighted_values = {
            a: airline_counts[a] * airline_distances[a] for a in airline_counts
        }

        # Calculate the total weighted value across all airlines in the row
        total = sum(weighted_values.values()) or 1  # avoid division by zero

        # Normalize the weighted values so the sum of the row equals 1
        normalized = {a: weighted_values.get(a, 0.0) / total for a in all_airlines}

        # Append the normalized values to the feature rows list
        feature_rows.append(normalized)
    
    # Convert the list of feature rows into a DataFrame
    feature_df = pd.DataFrame(feature_rows).fillna(0)
    return pd.concat([df.reset_index(drop=True), feature_df], axis=1)


df_train = airline_count_weighted_encoding(df_train)
df_test = airline_count_weighted_encoding(df_test)

### relBoing, relAirbus, relOther

In [46]:
def compute_weighted_aircraft_fractions(equipment_str, distance_str, total_distance):
    if not isinstance(equipment_str, str) or not isinstance(distance_str, str):
        return 0.0, 0.0, 0.0

    aircraft_list = equipment_str.split('||')
    distance_list = distance_str.split('||')

    try:
        distances = [float(d) for d in distance_list]
    except ValueError:
        return 0.0, 0.0, 0.0

    boeing_dist = sum(dist for a, dist in zip(aircraft_list, distances) if 'boeing' in a.lower())
    airbus_dist = sum(dist for a, dist in zip(aircraft_list, distances) if 'airbus' in a.lower())
    others_dist = total_distance - boeing_dist - airbus_dist

    rel_boeing = boeing_dist / total_distance
    rel_airbus = airbus_dist / total_distance
    rel_others = others_dist / total_distance

    return rel_boeing, rel_airbus, rel_others

def aircraft_relative_counts(df):
    # Apply the function 
    df['relBoeing'], df['relAirbus'], df['relOthers'] = zip(*df.apply(
        lambda row: compute_weighted_aircraft_fractions(
            row['segmentsEquipmentDescription'], 
            row['segmentsDistance'],
            row['total_distance']
        ), 
        axis=1
    ))

    return df

df_train = aircraft_relative_counts(df_train)
df_test = aircraft_relative_counts(df_test)

### isCoach

In [48]:
# Function to verify if at least one part of the trip is "coach"
def is_coach(cabin_code):
    parts = cabin_code.lower().split('||')
    return any(part.strip() == 'coach' for part in parts)

def is_coach_check(df):
    # Creating column isCoach:
    df['isCoach'] = df['segmentsCabinCode'].apply(is_coach)

    return df

df_train = is_coach_check(df_train)
df_test = is_coach_check(df_test)

### Convert boolean to binary encoding

In [50]:
def convert_boolean_to_numerical(df):
    df['isBasicEconomy'] = df['isBasicEconomy'].astype(int)
    df['isRefundable'] = df['isRefundable'].astype(int)
    df['isNonStop'] = df['isNonStop'].astype(int)
    df['isCoach'] = df['isCoach'].astype(int)

    return df

df_train = convert_boolean_to_numerical(df_train)
df_test = convert_boolean_to_numerical(df_test)

## Drop unusable features (Keep it in the end before normalization!!)

In [52]:
columns_to_drop = [
    'startingAirport',
    'fareBasisCode',
    'totalTravelDistance',
    'segmentsDepartureTimeEpochSeconds',
    'segmentsDepartureTimeRaw',
    'segmentsArrivalTimeEpochSeconds',
    'segmentsArrivalTimeRaw',
    'segmentsArrivalAirportCode',
    'segmentsDepartureAirportCode',
    'segmentsAirlineName',
    'segmentsAirlineCode',
    'segmentsEquipmentDescription',
    'segmentsDurationInSeconds',
    'segmentsDistance',
    'segmentsCabinCode',
    'isRefundable', 
    'total_distance_check',
    'longestDistanceAirline',
    'destinationAirport',
    'travelDuration',
    'searchDate', 
    'flightDate',
]

In [53]:
def drop_columns(df):
    df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')

    return df_cleaned

In [54]:
df_train = drop_columns(df_train)
df_test = drop_columns(df_test)

### SAVE FINAL TRAIN AND TEST

In [56]:
df_train.to_csv('preprocessed_train3.csv', index=False)
df_test.to_csv('preprocessed_test3.csv', index=False)

In [57]:
total_rows = len(df_train)
print(total_rows)

2689889
